In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, div.text_cell_render ol li p, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# OpenAI Chat Completions API 기본 (2025년 3월 기준)

이 튜토리얼은 OpenAI의 Chat Completions API를 활용하여 챗봇이나 AI 기능을 개발하는 방법을 단계별로 설명합니다. 특히 OpenAI의 최신 언어 모델 중 하나인 GPT-4o-mini/gpt-4.1-nano를 사용하여 예제를 진행할 것입니다. 각 섹션에는 개념 설명과 함께 실행 가능한 파이썬 코드 예제가 포함되어 있습니다.

### 주요 학습 내용:

1. OpenAI API 소개 및 환경 설정: OpenAI API 개요, API 키 발급 및 보안 설정, 파이썬 클라이언트 설치 및 인스턴스 생성 방법
2. 기본적인 Chat Completions API 사용법: 간단한 대화형 텍스트 생성 요청과 응답 처리, 프롬프트 엔지니어링 기초
3. 스트리밍 응답: 대화 응답을 스트리밍 방식으로 받아 실시간 처리하는 방법
4. 시스템 메시지 활용: 시스템 역할 메시지를 사용하여 AI의 응답 스타일이나 행동을 조정하는 방법
5. 고급 활용법: 토큰 최적화와 비용 절감 전략, OpenAI API 에러 처리 및 예외Handling
6. 실전 프로젝트 예제: 간단한 챗봇 구현 및 외부 데이터/API와 연동하여 데이터 분석 기능을 결합한 사례

## 1. OpenAI API 소개 및 환경 설정

먼저 OpenAI API와 Chat Completions에 대해 간략히 알아보고, API를 사용하기 위한 환경을 설정해보겠습니다.

### OpenAI API 개요
OpenAI API는 GPT 계열의 대규모 언어 모델을 인터넷을 통해 사용할 수 있도록 제공하는 서비스입니다. Chat Completions API는 챗봇과 유사한 대화형 상호작용을 할 수 있는 엔드포인트로, 역할(role)이 부여된 메시지 목록을 입력하면 모델이 다음 대화 내용을 생성합니다. GPT-4o는 2025년 3월 현재 가장 강력한 모델 중 하나로, 텍스트와 이미지 입력을 모두 처리하며 최대 128k 토큰의 긴 문맥을 다룰 수 있습니다. GPT-4o와 경량화 모델인 GPT-4o-mini 등이 제공되며, 요구 사항에 따라 적절한 모델을 선택할 수 있습니다 (GPT-4o-mini는 비용 효율이 높음)

### API 키 발급 및 보안 설정
OpenAI API를 사용하려면 먼저 OpenAI 계정에서 API 키를 발급받아야 합니다. OpenAI 웹사이트의 API Keys 페이지에서 새로운 비밀 키를 생성할 수 있습니다. 발급받은 API 키는 비밀로 관리해야 하며, 소스 코드나 공개 저장소에 노출되지 않도록 주의해야 합니다. 가장 좋은 방법은 API 키를 코드에 하드코딩하지 않고, 환경 변수나 별도의 설정 파일에 저장하는 것입니다. 이 튜토리얼에서는 .env 파일에 키를 저장하고 파이썬에서 이를 불러오는 방식을 사용합니다. 이를 위해 Python용 패키지 **python-dotenv**를 활용하겠습니다.

- .env 파일에 키 저장: 프로젝트 디렉터리에 .env 파일을 만들고 아래와 같이 API 키를 저장합니다 (따옴표 없이).

    ```
    OPENAI_API_KEY=발급받은-API키-값
    ```

- python-dotenv 사용: 파이썬 코드에서 python-dotenv를 이용해 .env 파일의 환경 변수를 불러올 수 있습니다.


In [2]:
import openai
openai.__version__

'2.9.0'

In [5]:
import os
from dotenv import load_dotenv
load_dotenv() # 환경변수 load
openai_key = os.getenv('OPENAI_API_KEY')
print('key :', openai_key[:3])

key : sk-


In [8]:
from openai import OpenAI
client = OpenAI()
# client = OpenAI(
    # api_key=openai_key
#)
response = client.responses.create(
    model="gpt-4.1-mini",
    input="웃긴 농담 하나 해줘"
)
response

Response(id='resp_09f92539646f25f50069390a42db4081949b2dada4f29c313d', created_at=1765345858.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_09f92539646f25f50069390a4382988194911a7c55c0eb34e6', content=[ResponseOutputText(annotations=[], text='물고기 두 마리가 길을 걷고 있었어요.  \n한 마리가 갑자기 벽에 부딪히자 다른 한 마리가 말했어요.  \n"야, 조심해! 벽이 있어!"  \n그러자 부딪힌 물고기가 대답했죠,  \n"뭐? 그게 뭔 소린지 물가게 해줘!" 😄', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, prompt_cache_retention=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='default', status='completed', text=ResponseTextConfig(f

In [10]:
response.output_text

'물고기 두 마리가 길을 걷고 있었어요.  \n한 마리가 갑자기 벽에 부딪히자 다른 한 마리가 말했어요.  \n"야, 조심해! 벽이 있어!"  \n그러자 부딪힌 물고기가 대답했죠,  \n"뭐? 그게 뭔 소린지 물가게 해줘!" 😄'

위 코드로 client 객체가 생성되었습니다. 이제 이 client를 통해 OpenAI Chat Completions API를 호출할 수 있습니다. 다음 섹션부터는 실제로 Chat Completions API를 호출하여 다양한 기능을 실습해보겠습니다.

## 2. 기본적인 Chat Completions API 사용법

이 섹션에서는 Chat Completions API를 사용하여 가장 기본적인 대화 생성 작업을 수행해봅니다.

### 간단한 텍스트 생성 요청
Chat Completions 엔드포인트는 메시지 목록을 입력으로 받아 다음에 이어질 메시지를 생성합니다. 각 메시지는 role과 content 필드로 구성되어 있으며, 일반적으로 **user (사용자 메시지), assistant (모델의 응답 메시지), system (시스템 지시 메시지)** 세 가지 역할을 사용합니다. 가장 간단한 예제로, 사용자 역할의 메시지 하나를 모델에 보내고 응답을 받아보겠습니다. 모델은 GPT-4o를 사용합니다.


In [11]:
# 사용자 메세지 구성
messages = [
    {"role":"user", "content":"""안녕하세요. 오늘 날씨가 어떤가요?"""}
]
response = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages,
    temperature=0.7, # 0~2 : 일관적~창의적(예측을 벗어난)
    frequency_penalty=0.5 # 값이 클수록 반복을 억제함 -2~2
)
response

ChatCompletion(id='chatcmpl-Cl7a3lt5gBysVELA6bUX6b47mcYpt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요! 죄송하지만, 저는 실시간 날씨 정보를 제공할 수 없습니다. 오늘의 날씨를 확인하시려면 기상청 홈페이지나 날씨 앱을 이용해 보시길 추천드립니다. 어떤 지역의 날씨를 알고 싶으신지 알려주시면, 일반적인 계절별 특징이나 참고할 만한 정보를 드릴 수도 있습니다!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1765346591, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_f0bc439dc3', usage=CompletionUsage(completion_tokens=77, prompt_tokens=18, total_tokens=95, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [15]:
response.choices[0].message.content

'안녕하세요! 죄송하지만, 저는 실시간 날씨 정보를 제공할 수 없습니다. 오늘의 날씨를 확인하시려면 기상청 홈페이지나 날씨 앱을 이용해 보시길 추천드립니다. 어떤 지역의 날씨를 알고 싶으신지 알려주시면, 일반적인 계절별 특징이나 참고할 만한 정보를 드릴 수도 있습니다!'

In [20]:
response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role":"system", "content":"너는 친절하게 대답해 주는 비서야"}, #역할부여
        {"role":"user",   "content":"2020년 월드 시리즈에서 누가 우승했어?"}, #질문
    ],
    temperature=1,
    frequency_penalty=0.5
)

In [21]:
response.usage

CompletionUsage(completion_tokens=51, prompt_tokens=39, total_tokens=90, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [22]:
response.choices[0].message.content

'2020년 월드 시리즈 우승팀은 로스앤젤레스 다저스(Los Angeles Dodgers)입니다. 그들은 탬파베이 레이스(Tampa Bay Rays)를 상대로 시리즈에서 승리했어요.'

In [ ]:
# 이전 답변을 포함하여 답변하기

In [23]:
response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role":"system", "content":"너는 친절하게 대답해 주는 비서야"}, #역할부여
        {"role":"user",   "content":"2020년 월드 시리즈에서 누가 우승했어?"}, #질문
        {"role":"assistant", 
         "content":"LA 다저스입니다. Tampa Bay Rays를 상대로 승리했어요."},
        {"role":"user", "content":"2020년 우승시 LA다저스가 몇대몇으로 승리했는지 간단하게 답해 주세요"}
    ],
    temperature=1,
    frequency_penalty=0.5
)
print(response.choices[0].message.content)

LA 다저스가 4승 2패로 승리했습니다.


In [24]:
# JSON형태로 받기
model = "gpt-4.1-mini"
response = client.chat.completions.create(
    model = model,
    messages=[
        {
            "role":"system", 
             "content":"You are a helpful assistant designed to output JSON"
        },
        {"role":"user", "content":"Who won the world series in 2020?"}
    ],
    response_format={"type":"json_object"} # json형태로 응답
)

In [26]:
result = response.choices[0].message.content
print(result)
print(type(result))

{
  "year": 2020,
  "world_series_champion": "Los Angeles Dodgers"
}
<class 'str'>


In [27]:
import json
dic_result = json.loads(result)
print(dic_result)
print(type(dic_result))

{'year': 2020, 'world_series_champion': 'Los Angeles Dodgers'}
<class 'dict'>


In [28]:
# 내일 -> 웹 버전 (입력된 데이터를 1줄 요약해서 보여주기)
from dotenv import load_dotenv
from openai import OpenAI
def askGPT(prompt):
    "GPT에게 요청하여 prompt내용을 1줄 요약하여 return"
    load_dotenv()
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {
             "role":"system", 
             "content":"당신은 한국어로 된 텍스트를 잘 요약하는 전문어시스턴트입니다"
            },
            {"role":"user", "content":prompt}            
        ]
    )
    return response.choices[0].message.content

In [ ]:
message = input("요약할 글을 입력하세요 :")
if message:
    prompt = f"""your task is to summarize the text sentences in Korean language.
    Summarize in 1 line. Use the format of a bullet point.
    text : {message}"""
    result = askGPT(prompt)
    print(result)